In [1]:
from pyspark.sql import SparkSession
from operator import add

spark_session = SparkSession.builder\
        .master("spark://192.168.2.250:7077") \
        .appName("samir_mehdiyev_A3")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores", 8)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# RDD API
spark_context = spark_session.sparkContext

spark_context.setLogLevel("INFO")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/19 16:45:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# A.1.1 Read the English transcripts with Spark, and count the number of lines.
rdd_english = spark_context.textFile("hdfs://192.168.2.250:9000/europarl/europarl-v7.sv-en.en")
english_line_count = rdd_english.count()
print(f"English line count: {english_line_count}")

# A.1.2 Do the same with the other language (so that you have a separate lineage of RDDs for each).
rdd_swedish = spark_context.textFile("hdfs://192.168.2.250:9000/europarl/europarl-v7.sv-en.sv")
swedish_line_count = rdd_swedish.count()
print(f"Swedish line count: {swedish_line_count}")

24/02/19 16:45:22 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 221.5 KiB, free 434.2 MiB)
24/02/19 16:45:22 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 32.6 KiB, free 434.2 MiB)
24/02/19 16:45:22 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on host-192-168-2-215-de1:10005 (size: 32.6 KiB, free: 434.4 MiB)
24/02/19 16:45:22 INFO SparkContext: Created broadcast 0 from textFile at NativeMethodAccessorImpl.java:0
24/02/19 16:45:23 INFO FileInputFormat: Total input files to process : 1
24/02/19 16:45:23 INFO NetworkTopology: Adding a new node: /default-rack/192.168.2.250:9866
24/02/19 16:45:23 INFO SparkContext: Starting job: count at /tmp/ipykernel_29018/3158762141.py:3
24/02/19 16:45:23 INFO DAGScheduler: Got job 0 (count at /tmp/ipykernel_29018/3158762141.py:3) with 2 output partitions
24/02/19 16:45:23 INFO DAGScheduler: Final stage: ResultStage 0 (count at /tmp/ipykernel_29018/3158762141.py:3)
24/02

English line count: 1862234


24/02/19 16:45:29 INFO BlockManagerInfo: Added broadcast_3_piece0 in memory on 192.168.2.252:10006 (size: 5.5 KiB, free: 434.4 MiB)
24/02/19 16:45:29 INFO BlockManagerInfo: Added broadcast_2_piece0 in memory on 192.168.2.252:10006 (size: 32.6 KiB, free: 434.3 MiB)
24/02/19 16:45:29 INFO TaskSetManager: Finished task 2.0 in stage 1.0 (TID 4) in 336 ms on 192.168.2.252 (executor 0) (1/3)


Swedish line count: 1862234


24/02/19 16:45:30 INFO TaskSetManager: Finished task 0.0 in stage 1.0 (TID 2) in 1478 ms on 192.168.2.252 (executor 0) (2/3)
24/02/19 16:45:30 INFO TaskSetManager: Finished task 1.0 in stage 1.0 (TID 3) in 1573 ms on 192.168.2.252 (executor 0) (3/3)
24/02/19 16:45:30 INFO TaskSchedulerImpl: Removed TaskSet 1.0, whose tasks have all completed, from pool 
24/02/19 16:45:30 INFO DAGScheduler: ResultStage 1 (count at /tmp/ipykernel_29018/3158762141.py:8) finished in 1.587 s
24/02/19 16:45:30 INFO DAGScheduler: Job 1 is finished. Cancelling potential speculative or zombie tasks for this job
24/02/19 16:45:30 INFO TaskSchedulerImpl: Killing all running tasks in stage 1: Stage finished
24/02/19 16:45:30 INFO DAGScheduler: Job 1 finished: count at /tmp/ipykernel_29018/3158762141.py:8, took 1.595876 s


In [3]:
#A.1.3 Verify that the line counts are the same for the two languages.
if english_line_count == swedish_line_count:
    print('Line counts match')
else:
    print('Line counts mismatch')

Line counts match


In [4]:
# A.1.4 Count the number of partitions.
english_partitions = rdd_english.getNumPartitions()
swedish_partitions = rdd_swedish.getNumPartitions()

print(f"English partitions: {english_partitions}")
print(f"Swedish partitions: {swedish_partitions}")

English partitions: 2
Swedish partitions: 3


In [5]:
# A.2.1 Pre-process the text from both RDDs
def pre_process_text(line):
    # Lowercase and tokenize
    return line.lower().split(" ")

english_preprocessed = rdd_english.map(pre_process_text)
swedish_preprocessed = rdd_swedish.map(pre_process_text)

In [6]:
# A.2.2 Inspect 10 entries from each of your RDDs to verify your pre-processing.
print("English sample:", english_preprocessed.take(10))
print("Swedish sample:", swedish_preprocessed.take(10))

24/02/19 16:45:31 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/02/19 16:45:31 INFO DAGScheduler: Got job 2 (runJob at PythonRDD.scala:181) with 1 output partitions
24/02/19 16:45:31 INFO DAGScheduler: Final stage: ResultStage 2 (runJob at PythonRDD.scala:181)
24/02/19 16:45:31 INFO DAGScheduler: Parents of final stage: List()
24/02/19 16:45:31 INFO DAGScheduler: Missing parents: List()
24/02/19 16:45:31 INFO DAGScheduler: Submitting ResultStage 2 (PythonRDD[6] at RDD at PythonRDD.scala:53), which has no missing parents
24/02/19 16:45:31 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 8.5 KiB, free 433.9 MiB)
24/02/19 16:45:31 INFO MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 5.2 KiB, free 433.9 MiB)
24/02/19 16:45:31 INFO BlockManagerInfo: Added broadcast_4_piece0 in memory on host-192-168-2-215-de1:10005 (size: 5.2 KiB, free: 434.3 MiB)
24/02/19 16:45:31 INFO SparkContext: Created broadcast 4 from b

English sample: [['resumption', 'of', 'the', 'session'], ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period.'], ['although,', 'as', 'you', 'will', 'have', 'seen,', 'the', 'dreaded', "'millennium", "bug'", 'failed', 'to', 'materialise,', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were', 'dreadful.'], ['you', 'have', 'requested', 'a', 'debate', 'on', 'this', 'subject', 'in', 'the', 'course', 'of', 'the', 'next', 'few', 'days,', 'during', 'this', 'part-session.'], ['in', 'the', 'meantime,', 'i', 'should', 'like', 'to', 'observe', 'a', "minute'", 's', 'silence,', 'as', 'a', 'number', 'of', 'members', 'have', 'requested,', 'on', '

Swedish sample: [['återupptagande', 'av', 'sessionen'], ['jag', 'förklarar', 'europaparlamentets', 'session', 'återupptagen', 'efter', 'avbrottet', 'den', '17', 'december.', 'jag', 'vill', 'på', 'nytt', 'önska', 'er', 'ett', 'gott', 'nytt', 'år', 'och', 'jag', 'hoppas', 'att', 'ni', 'haft', 'en', 'trevlig', 'semester.'], ['som', 'ni', 'kunnat', 'konstatera', 'ägde', '"den', 'stora', 'år', '2000-buggen"', 'aldrig', 'rum.', 'däremot', 'har', 'invånarna', 'i', 'ett', 'antal', 'av', 'våra', 'medlemsländer', 'drabbats', 'av', 'naturkatastrofer', 'som', 'verkligen', 'varit', 'förskräckliga.'], ['ni', 'har', 'begärt', 'en', 'debatt', 'i', 'ämnet', 'under', 'sammanträdesperiodens', 'kommande', 'dagar.'], ['till', 'dess', 'vill', 'jag', 'att', 'vi,', 'som', 'ett', 'antal', 'kolleger', 'begärt,', 'håller', 'en', 'tyst', 'minut', 'för', 'offren', 'för', 'bl.a.', 'stormarna', 'i', 'de', 'länder', 'i', 'europeiska', 'unionen', 'som', 'drabbats.'], ['jag', 'ber', 'er', 'resa', 'er', 'för', 'en', 'ty

24/02/19 16:45:32 INFO TaskSetManager: Finished task 0.0 in stage 3.0 (TID 6) in 1180 ms on 192.168.2.252 (executor 0) (1/1)
24/02/19 16:45:32 INFO TaskSchedulerImpl: Removed TaskSet 3.0, whose tasks have all completed, from pool 
24/02/19 16:45:32 INFO DAGScheduler: ResultStage 3 (runJob at PythonRDD.scala:181) finished in 1.200 s
24/02/19 16:45:32 INFO DAGScheduler: Job 3 is finished. Cancelling potential speculative or zombie tasks for this job
24/02/19 16:45:32 INFO TaskSchedulerImpl: Killing all running tasks in stage 3: Stage finished
24/02/19 16:45:32 INFO DAGScheduler: Job 3 finished: runJob at PythonRDD.scala:181, took 1.208704 s


In [7]:
# A.2.3 Verify that the line counts still match after the pre-processing.
english_preprocessed_line_count = english_preprocessed.count()
swedish_preprocessed_line_count = swedish_preprocessed.count()

if english_preprocessed_line_count == swedish_preprocessed_line_count:
    print('Line counts match')
else:
    print('Line counts mismatch')

24/02/19 16:45:32 INFO SparkContext: Starting job: count at /tmp/ipykernel_29018/4224381202.py:2
24/02/19 16:45:33 INFO DAGScheduler: Got job 4 (count at /tmp/ipykernel_29018/4224381202.py:2) with 2 output partitions
24/02/19 16:45:33 INFO DAGScheduler: Final stage: ResultStage 4 (count at /tmp/ipykernel_29018/4224381202.py:2)
24/02/19 16:45:33 INFO DAGScheduler: Parents of final stage: List()
24/02/19 16:45:33 INFO DAGScheduler: Missing parents: List()
24/02/19 16:45:33 INFO DAGScheduler: Submitting ResultStage 4 (PythonRDD[8] at count at /tmp/ipykernel_29018/4224381202.py:2), which has no missing parents
24/02/19 16:45:33 INFO MemoryStore: Block broadcast_6 stored as values in memory (estimated size 9.7 KiB, free 433.9 MiB)
24/02/19 16:45:33 INFO MemoryStore: Block broadcast_6_piece0 stored as bytes in memory (estimated size 5.7 KiB, free 433.8 MiB)
24/02/19 16:45:33 INFO BlockManagerInfo: Added broadcast_6_piece0 in memory on host-192-168-2-215-de1:10005 (size: 5.7 KiB, free: 434.3 

Line counts match


24/02/19 16:45:37 INFO TaskSetManager: Finished task 1.0 in stage 5.0 (TID 10) in 2766 ms on 192.168.2.252 (executor 0) (2/3)
24/02/19 16:45:37 INFO TaskSetManager: Finished task 0.0 in stage 5.0 (TID 9) in 2850 ms on 192.168.2.252 (executor 0) (3/3)
24/02/19 16:45:37 INFO TaskSchedulerImpl: Removed TaskSet 5.0, whose tasks have all completed, from pool 
24/02/19 16:45:37 INFO DAGScheduler: ResultStage 5 (count at /tmp/ipykernel_29018/4224381202.py:3) finished in 2.872 s
24/02/19 16:45:37 INFO DAGScheduler: Job 5 is finished. Cancelling potential speculative or zombie tasks for this job
24/02/19 16:45:37 INFO TaskSchedulerImpl: Killing all running tasks in stage 5: Stage finished
24/02/19 16:45:37 INFO DAGScheduler: Job 5 finished: count at /tmp/ipykernel_29018/4224381202.py:3, took 2.880994 s


In [8]:
# A.3.1 Use Spark to compute the 10 most frequently according words in the English language corpus. Repeat for the other language.
def get_top_words(rdd):
    return rdd.flatMap(lambda x: x).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b).takeOrdered(10, key=lambda x: -x[1])

english_top_words = get_top_words(english_preprocessed)
swedish_top_words = get_top_words(swedish_preprocessed)

print("Top 10 English words:", english_top_words)
print("Top 10 Swedish words:", swedish_top_words)


24/02/19 16:45:38 INFO SparkContext: Starting job: takeOrdered at /tmp/ipykernel_29018/1648545257.py:3
24/02/19 16:45:38 INFO DAGScheduler: Registering RDD 11 (reduceByKey at /tmp/ipykernel_29018/1648545257.py:3) as input to shuffle 0
24/02/19 16:45:38 INFO DAGScheduler: Got job 6 (takeOrdered at /tmp/ipykernel_29018/1648545257.py:3) with 2 output partitions
24/02/19 16:45:38 INFO DAGScheduler: Final stage: ResultStage 7 (takeOrdered at /tmp/ipykernel_29018/1648545257.py:3)
24/02/19 16:45:38 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 6)
24/02/19 16:45:38 INFO DAGScheduler: Missing parents: List(ShuffleMapStage 6)
24/02/19 16:45:38 INFO DAGScheduler: Submitting ShuffleMapStage 6 (PairwiseRDD[11] at reduceByKey at /tmp/ipykernel_29018/1648545257.py:3), which has no missing parents
24/02/19 16:45:38 INFO MemoryStore: Block broadcast_8 stored as values in memory (estimated size 13.2 KiB, free 433.8 MiB)
24/02/19 16:45:38 INFO MemoryStore: Block broadcast_8_piece0 store

Top 10 English words: [('the', 3498375), ('of', 1659758), ('to', 1539760), ('and', 1288401), ('in', 1085993), ('that', 797516), ('a', 773522), ('is', 758050), ('for', 534242), ('we', 522849)]
Top 10 Swedish words: [('att', 1706293), ('och', 1344830), ('i', 1050774), ('det', 924866), ('som', 913276), ('för', 908680), ('av', 738068), ('är', 694381), ('en', 620310), ('vi', 539797)]


24/02/19 16:46:10 INFO TaskSetManager: Finished task 1.0 in stage 9.0 (TID 20) in 285 ms on 192.168.2.252 (executor 0) (1/3)
24/02/19 16:46:10 INFO TaskSetManager: Finished task 0.0 in stage 9.0 (TID 19) in 290 ms on 192.168.2.252 (executor 0) (2/3)
24/02/19 16:46:10 INFO TaskSetManager: Finished task 2.0 in stage 9.0 (TID 21) in 313 ms on 192.168.2.252 (executor 0) (3/3)
24/02/19 16:46:10 INFO TaskSchedulerImpl: Removed TaskSet 9.0, whose tasks have all completed, from pool 
24/02/19 16:46:10 INFO DAGScheduler: ResultStage 9 (takeOrdered at /tmp/ipykernel_29018/1648545257.py:3) finished in 0.346 s
24/02/19 16:46:10 INFO DAGScheduler: Job 7 is finished. Cancelling potential speculative or zombie tasks for this job
24/02/19 16:46:10 INFO TaskSchedulerImpl: Killing all running tasks in stage 9: Stage finished
24/02/19 16:46:10 INFO DAGScheduler: Job 7 finished: takeOrdered at /tmp/ipykernel_29018/1648545257.py:3, took 15.746721 s


In [9]:
# A.3.2 Verify that your results are reasonable.
# Results look like correct, at least for english it is mostly correct that these are top frequent words

In [10]:
# A.4.1 Use this parallel corpus to mine some translations in the form of word pairs, for the two
# languages. Do this by pairing words found on short lines with the same number of words
# respectively. We (incorrectly) assume the words stay in the same order when translated.

# Key the lines by their line number
english_indexed = english_preprocessed.zipWithIndex()
swedish_indexed = swedish_preprocessed.zipWithIndex()

# Swap the key and value - so that the line number is the key.
english_swapped = english_indexed.map(lambda x: (x[1], x[0]))
swedish_swapped = swedish_indexed.map(lambda x: (x[1], x[0]))

# Join the two RDDs on the line number
joined_rdd = english_swapped.join(swedish_swapped)

# Filter to exclude line pairs that have an empty/missing “corresponding” sentence.
filtered_non_empty = joined_rdd.filter(lambda x: x[1][0] and x[1][1])

# Filter to leave only pairs of sentences with a small number of words per sentence
filtered_short_sentences = filtered_non_empty.filter(lambda x: len(x[1][0]) <= 5 and len(x[1][1]) <= 5)

# Filter to leave only pairs of sentences with the same number of words in each sentence.
filtered_equal_words = filtered_short_sentences.filter(lambda x: len(x[1][0]) == len(x[1][1]))

# For each sentence pair, map so that you pair each (in order) word in the two sentences. I add isalpha() check in order to get only words.
word_pairs = word_pairs = filtered_equal_words.flatMap(
    lambda x: [(en, sv) for en, sv in zip(x[1][0], x[1][1]) if en.isalpha() and sv.isalpha()]
)

# Use reduce to count the number of occurrences of the word-translation-pairs.
word_pair_counts = word_pairs.map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b)

# Print some of the most frequently occurring pairs of words.
most_frequent_pairs = word_pair_counts.takeOrdered(10, key=lambda x: -x[1])
print(most_frequent_pairs)


24/02/19 16:46:10 INFO SparkContext: Starting job: zipWithIndex at /tmp/ipykernel_29018/584516410.py:6
24/02/19 16:46:10 INFO DAGScheduler: Got job 8 (zipWithIndex at /tmp/ipykernel_29018/584516410.py:6) with 2 output partitions
24/02/19 16:46:10 INFO DAGScheduler: Final stage: ResultStage 10 (zipWithIndex at /tmp/ipykernel_29018/584516410.py:6)
24/02/19 16:46:10 INFO DAGScheduler: Parents of final stage: List()
24/02/19 16:46:10 INFO DAGScheduler: Missing parents: List()
24/02/19 16:46:10 INFO DAGScheduler: Submitting ResultStage 10 (PythonRDD[20] at zipWithIndex at /tmp/ipykernel_29018/584516410.py:6), which has no missing parents
24/02/19 16:46:10 INFO MemoryStore: Block broadcast_12 stored as values in memory (estimated size 8.4 KiB, free 433.9 MiB)
24/02/19 16:46:10 INFO MemoryStore: Block broadcast_12_piece0 stored as bytes in memory (estimated size 5.1 KiB, free 433.9 MiB)
24/02/19 16:46:10 INFO BlockManagerInfo: Added broadcast_12_piece0 in memory on host-192-168-2-215-de1:1000

[(('is', 'är'), 4699), (('that', 'det'), 1494), (('we', 'vi'), 1443), (('the', 'jag'), 1336), (('is', 'debatten'), 1327), (('debate', 'förklarar'), 1319), (('the', 'debatten'), 1253), (('is', 'härmed'), 1240), (('debate', 'är'), 1211), (('i', 'jag'), 1168)]


24/02/19 16:47:21 INFO TaskSetManager: Finished task 2.0 in stage 13.0 (TID 34) in 27669 ms on 192.168.2.252 (executor 0) (5/5)
24/02/19 16:47:21 INFO TaskSchedulerImpl: Removed TaskSet 13.0, whose tasks have all completed, from pool 
24/02/19 16:47:21 INFO DAGScheduler: ShuffleMapStage 13 (reduceByKey at /tmp/ipykernel_29018/584516410.py:31) finished in 27.685 s
24/02/19 16:47:21 INFO DAGScheduler: looking for newly runnable stages
24/02/19 16:47:21 INFO DAGScheduler: running: Set()
24/02/19 16:47:21 INFO DAGScheduler: waiting: Set(ResultStage 14)
24/02/19 16:47:21 INFO DAGScheduler: failed: Set()
24/02/19 16:47:21 INFO DAGScheduler: Submitting ResultStage 14 (PythonRDD[33] at takeOrdered at /tmp/ipykernel_29018/584516410.py:34), which has no missing parents
24/02/19 16:47:21 INFO MemoryStore: Block broadcast_16 stored as values in memory (estimated size 11.3 KiB, free 433.8 MiB)
24/02/19 16:47:21 INFO MemoryStore: Block broadcast_16_piece0 stored as bytes in memory (estimated size 6.

In [11]:
spark_context.stop()

24/02/19 16:55:35 INFO SparkContext: SparkContext is stopping with exitCode 0.
24/02/19 16:55:35 INFO SparkUI: Stopped Spark web UI at http://host-192-168-2-215-de1:4040
24/02/19 16:55:35 INFO StandaloneSchedulerBackend: Shutting down all executors
24/02/19 16:55:35 INFO StandaloneSchedulerBackend$StandaloneDriverEndpoint: Asking each executor to shut down
24/02/19 16:55:35 INFO MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
24/02/19 16:55:35 INFO MemoryStore: MemoryStore cleared
24/02/19 16:55:35 INFO BlockManager: BlockManager stopped
24/02/19 16:55:35 INFO BlockManagerMaster: BlockManagerMaster stopped
24/02/19 16:55:35 INFO OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
24/02/19 16:55:35 INFO SparkContext: Successfully stopped SparkContext
